In [1]:
import pandas as pd
from tqdm import tqdm
import pubchempy as pcp
import numpy as np
from gensim.models import Word2Vec
from gensim.models import word2vec
from mol2vec import features
from mol2vec import helpers
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from rdkit import Chem
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [2]:
node2drugid = pd.read_csv('nodeidx2drugid.csv')

In [3]:
node2drugid.shape

(4267, 2)

# Get Mol2Vec

In [4]:
DrugBank = pd.read_pickle('DB_df_clean_PubChem_Fetch.pkl')

In [5]:
drug_id_smile = dict()

for index, row in tqdm(DrugBank.iterrows()):
    drug_id_smile[row['drug_id']] = row['drug_smiles']

25579it [00:01, 15677.91it/s]


In [8]:
SMILES_list = []

for index, row in tqdm(node2drugid.iterrows()):
    try:
        SMILES_list.append(drug_id_smile[row['drug id']])
    except:
        SMILES_list.append('NA')

4267it [00:00, 17521.08it/s]


In [9]:
node2drugid['SMILES'] = SMILES_list

In [10]:
node2drugid.head(5)

,node idx,drug id,SMILES
0,0,DB00001,NA
1,1,DB00002,NA
2,2,DB00004,NA
3,3,DB00005,NA
4,4,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...


In [13]:
node2drugid_valid_smile = node2drugid[node2drugid['SMILES'] != 'NA']

In [24]:
node2drugid_valid_smile = node2drugid_valid_smile.reset_index()

In [32]:
mol2vec_model = word2vec.Word2Vec.load('model_300dim.pkl')

drug_smiles = node2drugid['SMILES']
embedding_dimension = 300

# Create empty array to hold embeddings
drug_embeddings = np.zeros((len(node2drugid), embedding_dimension))
miss_words = []
hit_words = 0
bad_mol = 0
percent_unknown = []

for idx, row in tqdm(node2drugid.iterrows()):
    
    if row['SMILES'] != 'NA':

        molecule = Chem.MolFromSmiles(row['SMILES'])
        try:
            # Get fingerprint from molecule
            sub_structures = mol2alt_sentence(molecule, 2)
        except: 
            continue    

        # Iterate over each sub structure
        for sub in sub_structures:
            # Check to see if substructure exists
            try:
                drug_embeddings[idx, :] = drug_embeddings[idx, :] + mol2vec_model.wv[sub]
            
            # If not, replace with UNK (unknown)
            except:
                drug_embeddings[idx, :] = drug_embeddings[idx, :] + mol2vec_model.wv['UNK']
                
    else:
        
        drug_embeddings[idx,:] = mol2vec_model.wv['UNK']
 

4267it [00:02, 1776.05it/s]


In [36]:
np.save('mol2vec_embeddings.npy',drug_embeddings)

In [46]:
drug_embeddings.shape

(4267, 300)

# Load Data and Run Analysis

In [4]:
drug_embeddings = np.load('mol2vec_embeddings.npy')

## K = 5

In [38]:
k_clusters = 5
mol2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings)
davies_bouldin_score(drug_embeddings,mol2vec_kmeans.labels_)

0.899885672854192

In [41]:
node2vec = np.load('ddi_node2vec.npy')
node2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(node2vec)
davies_bouldin_score(node2vec,node2vec_kmeans.labels_)

2.2278830655072084

In [42]:
def crazyshuffle(arr):
    
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    
    return arr[rows, cols]

In [50]:
drug_embeddings = crazyshuffle(drug_embeddings)
shuffled_mol2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings)
davies_bouldin_score(drug_embeddings,shuffled_mol2vec_kmeans.labels_)

1.3902685736032687

In [51]:
drug_embeddings_random = np.random.rand(drug_embeddings.shape[0],drug_embeddings.shape[1])
drug_embeddings_random_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings_random)
davies_bouldin_score(drug_embeddings_random,drug_embeddings_random_kmeans.labels_)

12.946352761193154

In [55]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, node2vec_kmeans.labels_)

1.0

In [52]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, mol2vec_kmeans.labels_)

0.08094498522004298

In [53]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, shuffled_mol2vec_kmeans.labels_)

0.0001882428686888402

In [54]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, drug_embeddings_random_kmeans.labels_)

-0.00023844962394886939

## K = 10

In [5]:
k_clusters = 10
mol2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings)
davies_bouldin_score(drug_embeddings,mol2vec_kmeans.labels_)

1.1377191292579503

In [6]:
node2vec = np.load('ddi_node2vec.npy')
node2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(node2vec)
davies_bouldin_score(node2vec,node2vec_kmeans.labels_)

2.490250186462725

In [7]:
def crazyshuffle(arr):
    
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    
    return arr[rows, cols]

In [63]:
drug_embeddings = crazyshuffle(drug_embeddings)
shuffled_mol2vec_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings)
davies_bouldin_score(drug_embeddings,shuffled_mol2vec_kmeans.labels_)

0.7759653134050797

In [9]:
drug_embeddings_random = np.random.rand(drug_embeddings.shape[0],drug_embeddings.shape[1])
drug_embeddings_random_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(drug_embeddings_random)
davies_bouldin_score(drug_embeddings_random,drug_embeddings_random_kmeans.labels_)

10.962253512408205

In [10]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, node2vec_kmeans.labels_)

1.0

In [11]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, mol2vec_kmeans.labels_)

0.07049575908224975

In [12]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, shuffled_mol2vec_kmeans.labels_)

-3.588414796679684e-05

In [13]:
adjusted_mutual_info_score(node2vec_kmeans.labels_, drug_embeddings_random_kmeans.labels_)

-0.0001897953193499187